In [10]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from model.lanenet.LaneNet import LaneNet
from PIL import Image

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [11]:
def preprocess_frame(frame, transform):
    frame = Image.fromarray(frame)
    frame = transform(frame)
    return frame

def detect_lanes(binary_pred, num_windows=10, margin=100, minpix=50):
    height, width = binary_pred.shape
    
    # ROI 설정 (하단 60%)
    roi_height = int(height * 0.6)
    roi = binary_pred[height - roi_height:, :]
    
    # 히스토그램으로 시작점 찾기
    histogram = np.sum(roi[roi.shape[0]//2:, :], axis=0)
    midpoint = width // 2
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    window_height = roi_height // num_windows
    nonzero = roi.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(num_windows):
        win_y_low = roi_height - (window + 1) * window_height
        win_y_high = roi_height - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
    
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] + (height - roi_height)
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] + (height - roi_height)
    
    left_fit = None
    right_fit = None
    if len(leftx) > 0:
        left_fit = np.polyfit(lefty, leftx, 1)
    if len(rightx) > 0:
        right_fit = np.polyfit(righty, rightx, 1)
    
    return left_fit, right_fit

In [12]:
def draw_lanes(frame, left_fit, right_fit):
    height, width, _ = frame.shape
    ploty = np.linspace(height - int(height * 0.6), height - 1, num=int(height * 0.6))
    
    if left_fit is not None:
        left_fitx = left_fit[0] * ploty + left_fit[1]
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        cv2.polylines(frame, [pts_left.astype(np.int32)], isClosed=False, color=(255, 0, 0), thickness=5)
    
    if right_fit is not None:
        right_fitx = right_fit[0] * ploty + right_fit[1]
        pts_right = np.array([np.transpose(np.vstack([right_fitx, ploty]))])
        cv2.polylines(frame, [pts_right.astype(np.int32)], isClosed=False, color=(0, 0, 255), thickness=5)
    
    return frame

In [13]:
def test_with_webcam():
    model_path = r'C:\Users\yth12\Dropbox\4. 기타 자료\해군 AI 특강\AI_Lecture\Day1\Deep-Learning\LaneNet\log\lanenet_DeepLabv3+_CrossEntrophy_epoch100_batchsize8.pth'
    resize_height, resize_width = 480, 640

    data_transform = transforms.Compose([
        transforms.Resize((resize_height, resize_width)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    model = LaneNet(arch='DeepLabv3+')
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(DEVICE)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        input_img = preprocess_frame(frame, data_transform).to(DEVICE)
        input_img = torch.unsqueeze(input_img, dim=0)

        with torch.no_grad():
            outputs = model(input_img)

        binary_pred = torch.squeeze(outputs['binary_seg_pred']).to('cpu').numpy() * 255
        
        # 차선 검출
        left_fit, right_fit = detect_lanes(binary_pred)
        
        # 검출된 차선 그리기
        result_frame = draw_lanes(frame, left_fit, right_fit)

        cv2.imshow('Lane Detection Result', result_frame)
        cv2.imshow('Binary Segmentation', binary_pred.astype(np.uint8))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
if __name__ == "__main__":
    test_with_webcam()

Use DeepLabv3+ as backbone


C:\Users\yth12\AppData\Local\Temp\ipykernel_21808\2868002389.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
C:\Users\yth12\AppData\